In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as LSCm
from scipy.interpolate import griddata
from scipy.signal import savgol_filter
from scipy import integrate


In [2]:
from NesrHydrusAnalyst import *

In [3]:
src = '../Datasets/sample3d'

# Modifying data grabbing

In [4]:
# tests get_grid_values
data_frame= read_hydrus_data(folder=src, save_to_csv=False)

In [5]:
v=0
X, Z, M, x_vals, z_vals = get_grid_values(data_frame,variable=v)

In [6]:
# print('x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape, 
#                                                  z_vals.shape, X.shape, 
#                                                  Z.shape, M.shape))

In [7]:
get_available_timesteps(data_frame)

[0, 15, 60, 120, 180, 360, 720, 1440]

In [8]:
get_full_dimensions(data_frame)

{'x': (0.0, 70.0), 'y': (0.0, 100.0), 'z': (-125.0, 0.0)}

In [9]:
get_legend_range(M.min(), M.max())

array([0.230, 0.240, 0.250, 0.260, 0.270, 0.280, 0.290, 0.300, 0.310,
       0.320, 0.330, 0.340, 0.350])

In [10]:
get_one_line_df(src, simulation_name="Sand Ditch simulation", dims='3d').T

,SimulTime_s,L_Unit,T_Unit,Category,MaxIt,TolTh,TolH,InitH/W,lWat,lChem,...,WatBalT720,WatBalT1440,hMean0,hMean15,hMean60,hMean120,hMean180,hMean360,hMean720,hMean1440
Sand Ditch simulation,209.75,1.0,1.0,3.0,10.0,0.001,1.0,1.0,1.0,0.0,...,65.245,124.33,-10.221,-10.229,-10.262,-10.306,-10.344,-11.299,-13.464,-16.12


In [12]:
data_frame.head()

,n,x,y,z,Th_T=0,Th_T=15,Th_T=60,Th_T=120,Th_T=180,Th_T=360,Th_T=720,Th_T=1440,H_T=0,H_T=15,H_T=60,H_T=120,H_T=180,H_T=360,H_T=720,H_T=1440
1,1,2.91667,0.0,-2.91667,0.210063,0.191791,0.158732,0.139777,0.130033,0.113877,0.100160,0.089611,-10.2125,-11.2717,-13.5745,-15.3043,-16.5198,-18.8943,-21.5512,-24.7883
2,2,2.91667,0.0,-5.83333,0.210063,0.203564,0.172109,0.152735,0.140227,0.122310,0.107456,0.094662,-10.2125,-10.5847,-12.4606,-14.0738,-15.2490,-17.4841,-20.1368,-23.2386
3,3,70.00000,0.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210067,0.203415,0.199713,0.167566,-10.2125,-10.2125,-10.2125,-10.2125,-10.2123,-10.6090,-10.8127,-12.8261
4,4,0.00000,0.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210071,0.194964,0.198825,0.167340,-10.2125,-10.2125,-10.2125,-10.2125,-10.2120,-11.0986,-10.8698,-12.8375
5,5,70.00000,100.0,-125.00000,0.210063,0.210063,0.210063,0.210063,0.210067,0.194068,0.199096,0.167400,-10.2125,-10.2125,-10.2125,-10.2125,-10.2122,-11.1508,-10.8534,-12.8323


# Get data from Sand Ditch 3D simulation

In [13]:
src = '../Datasets/H3D2_SandDitch0011'

In [14]:
df= read_hydrus_data(folder=src, save_to_csv=False)

In [16]:
df.head()

,n,x,y,z,Th_T=0,Th_T=5,Th_T=15,Th_T=30,Th_T=60,Th_T=120,...,H_T=5,H_T=15,H_T=30,H_T=60,H_T=120,H_T=180,H_T=240,H_T=500,H_T=720,H_T=1440
1,1,0.0000,0.0,0.00000,0.15,0.056079,0.056170,0.055658,0.054891,0.382578,...,-57.6896,-57.40750,-59.03960,-61.730800,-4.28025,-4.77654,-5.51407,-7.66429,-8.79789,-10.01450
2,2,49.9601,0.0,1.99787,0.15,0.053222,0.053808,0.053949,0.053876,0.266495,...,-68.9306,-66.15350,-65.53300,-65.850000,-8.03020,-6.91749,-7.33861,-9.28291,-9.97427,-9.55989
3,3,49.8802,0.0,3.99627,0.15,0.203979,0.208893,0.211409,0.213525,0.359747,...,-143.6060,-129.86400,-123.52800,-118.526000,-10.08760,-8.96240,-9.38220,-11.33150,-12.00630,-11.55600
4,4,27.3981,0.0,3.09723,0.15,0.245272,0.261515,0.270286,0.390000,0.381931,...,-67.8090,-52.53410,-45.99290,4.978220,-4.42541,-5.89492,-6.76845,-8.86218,-9.78174,-10.86720
5,5,26.6789,0.0,21.08290,0.10,0.072666,0.394500,0.417685,0.430000,0.202791,...,-33.3201,-3.84076,-2.82451,-0.922182,-10.76650,-12.89320,-14.40420,-18.55390,-20.73460,-24.69590


In [ ]:
v = 0
X, Z, M, x_vals, z_vals = get_grid_values(data_frame, variable=v)

display(get_available_timesteps(df),
        get_full_dimensions(data_frame),
        get_legend_range(M.min(), M.max()),
        'x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape,
                                                   z_vals.shape, X.shape,
                                                   Z.shape, M.shape))